In [ ]:
!pip install collections

In [1]:
import random
from collections import defaultdict

# Simulated Question Bank (60 questions)
questions = [
    {
        "id": i,
        "topic": f"Topic {topic}",
        "difficulty": difficulty,
        "text": f"{difficulty} question {i//12 + 1} for Topic {topic}"
    }
    for topic in range(1, 6)
    for i, difficulty in enumerate(
        ["Easy"]*4 + ["Medium"]*4 + ["Hard"]*4,
        start=(topic-1)*12
    )
]

class CATSystem:
    def __init__(self):
        self.user_performance = defaultdict(list)
        self.current_test = 1

    def run_test(self):
        if self.current_test == 1:
            test_questions = self._select_initial_questions()
        else:
            test_questions = self._select_adaptive_questions()

        print(f"\n=== Starting Test {self.current_test} ===")
        results = self._simulate_test_taking(test_questions)
        self._analyze_performance(results)

        self.current_test += 1
        return results

    def _select_initial_questions(self):
        """Select 4 easy questions from each topic (20 total)"""
        selected = []
        for topic in range(1, 6):
            topic_questions = [q for q in questions
                             if q["topic"] == f"Topic {topic}"
                             and q["difficulty"] == "Easy"][:4]
            selected.extend(topic_questions)
        return selected

    def _select_adaptive_questions(self):
        """Select questions based on previous performance"""
        selected = []
        for topic in range(1, 6):
            topic_perf = self.user_performance[f"Topic {topic}"]

            # Get all questions from previous test in this topic
            prev_questions = [q for q in self.user_performance["Test1"]
                            if q["question"]["topic"] == f"Topic {topic}"]

            # Check if all answers were correct
            if all(res["correct"] for res in prev_questions):
                difficulty = "Medium"
            else:
                difficulty = "Easy"

            # Select 4 questions of appropriate difficulty
            topic_questions = [q for q in questions
                             if q["topic"] == f"Topic {topic}"
                             and q["difficulty"] == difficulty][:4]
            selected.extend(topic_questions)
        return selected

    def _simulate_test_taking(self, test_questions):
        """Simulate user answering questions with random performance"""
        results = []
        for question in test_questions:
            # Simulate user response (80% chance correct for demonstration)
            is_correct = random.random() < 0.8  # Adjust this for different outcomes
            time_spent = round(random.uniform(10, 60), 2)  # 10-60 seconds

            result = {
                "question": question,
                "correct": is_correct,
                "time_spent": time_spent,
                "attempts": 1  # Simulating single attempt
            }
            results.append(result)

            # Track performance per topic
            self.user_performance[question["topic"]].append(result)

        return results

    def _analyze_performance(self, results):
        """Analyze and display test results"""
        correct = sum(1 for r in results if r["correct"])
        print(f"\nTest {self.current_test} Results:")
        print(f"Total Correct: {correct}/{len(results)}")

        # Display topic-wise performance
        topic_perf = defaultdict(list)
        for res in results:
            topic_perf[res["question"]["topic"]].append(res["correct"])

        print("\nTopic Performance:")
        for topic, scores in topic_perf.items():
            correct = sum(scores)
            print(f"{topic}: {correct}/{len(scores)} correct")

    def get_upgraded_topics(self):
        """Get list of topics that will get harder questions"""
        upgraded = []
        for topic in range(1, 6):
            prev_results = self.user_performance.get(f"Topic {topic}", [])
            if prev_results and all(res["correct"] for res in prev_results):
                upgraded.append(f"Topic {topic}")
        return upgraded

# Simulation Usage
if __name__ == "__main__":
    cat = CATSystem()

    # Take Test 1
    test1_results = cat.run_test()

    # Show which topics will be upgraded
    upgraded_topics = cat.get_upgraded_topics()
    print("\nUpgraded Topics for Test 2:", upgraded_topics or "None")

    # Take Test 2
    test2_results = cat.run_test()


=== Starting Test 1 ===

Test 1 Results:
Total Correct: 14/20

Topic Performance:
Topic 1: 1/4 correct
Topic 2: 4/4 correct
Topic 3: 4/4 correct
Topic 4: 2/4 correct
Topic 5: 3/4 correct

Upgraded Topics for Test 2: ['Topic 2', 'Topic 3']

=== Starting Test 2 ===

Test 2 Results:
Total Correct: 15/20

Topic Performance:
Topic 1: 4/4 correct
Topic 2: 4/4 correct
Topic 3: 2/4 correct
Topic 4: 3/4 correct
Topic 5: 2/4 correct
